### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

1. 18-25 is by far the biggest bucket of customers. Related, age is a better indicator of purchasing frequency than Gender.
2. The higher priced items appear to be more profitable than the bigger sellers
3. 35-39 showed the most purchasing power overall on a per transaction basis, almost 20% over the nearest other age group


In [21]:
import pandas as pd
import numpy as np 

heroes = "Resources/purchase_data.csv"

hero_data = pd.read_csv(heroes, encoding='utf-8')
hero_columns = hero_data[['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price', 'Purchase ID']]
hero_new = hero_columns.rename(columns={'SN': 'User Name'})
hero_new['Gender'] = hero_new['Gender'].replace(
    {'Other / Non-Disclosed': 'Other'})
hero_new = hero_new.set_index("Purchase ID")
hero_new.tail()

,User Name,Age,Gender,Item ID,Item Name,Price
Purchase ID,,,,,,
775,Aethedru70,21,Female,60,Wolf,3.54
776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,Sisur91,7,Male,101,Final Critic,4.19
779,Ennrian78,24,Male,50,Dawn,4.60


## Player Count

* Display the total number of players

In [22]:
players = len(hero_new['User Name'].value_counts())
total = pd.DataFrame({"Total Players": players}, index=[0])
total

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [23]:
avg_price = hero_new['Price'].mean()
total_sales = hero_new['Price'].sum()
items = hero_new['Item ID'].sum()

addition = hero_new['User Name'].value_counts() + hero_new['Price'].mean() + hero_new['Price'].sum() + hero_new['Item ID'].sum()

summary = pd.DataFrame({"Average Sale": [avg_price],
                              "Total Sales": [total_sales],
                              "Total Items Sold": [items]})
summary["Average Sale"] = summary["Average Sale"].astype(float).map(
    "${:,.2f}".format)
summary["Total Sales"] = summary["Total Sales"].astype(float).map(
    "${:,.2f}".format)
summary["Total Items Sold"] = summary["Total Items Sold"].astype(float).map(
    "{:,.0f}".format)

summary

,Average Sale,Total Sales,Total Items Sold
0,$3.05,"$2,379.77","71,849"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [24]:
genders = hero_new[['Gender', 'User Name']]
genders = genders.drop_duplicates()

gen_ct = genders['Gender'].value_counts()
gen_ct = [gen_ct[0],gen_ct[1],gen_ct[2]]
gen_pct = [round((gen_ct[0]/players) * 100,2),round((gen_ct[1]/players) * 100,2),round((gen_ct[2]/players) * 100,2)]

gen = pd.DataFrame({ 'Percentage of Players': gen_pct,'Total Count': gen_ct})
gen['Percentage of Players'] = gen['Percentage of Players'].astype(float).map('{:.2f}%'.format)
gen.index = (['Male', 'Female', 'Other'])

gen

,Percentage of Players,Total Count
Male,84.03%,484
Female,14.06%,81
Other,1.91%,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [25]:
genders = genders.drop_duplicates()
genders = hero_new.groupby(['Gender'])

pur_ct = genders['User Name'].count()

pur_price = genders['Price'].mean()

pur_val = genders['Price'].sum()

dups = hero_new.drop_duplicates(subset='User Name', keep='first')
group_dups = dups.groupby(['Gender'])


norms = (genders['Price'].sum() / group_dups['User Name'].count())


pur_df = pd.DataFrame({'Purchase Count': pur_ct,
                              'Average Purchase Price': pur_price,
                              'Total Purchase Value': pur_val,
                              'Normalized Totals': norms})

pur_df["Average Purchase Price"] = pur_df["Average Purchase Price"].map("${:.2f}".format)
pur_df["Total Purchase Value"] = pur_df["Total Purchase Value"].map("${:,.2f}".format)
pur_df["Normalized Totals"] = pur_df["Normalized Totals"].map("${:.2f}".format)
puf_df = pur_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
pur_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [26]:
bins = [0, 17.9, 25.9, 34.9, 39.9, 99]
group_names = ['Under 18', '18-25', '26-34', '35-39', '40+']
pd.cut(hero_new["Age"], bins, labels=group_names)
hero_new["Age Group"] = pd.cut(hero_new["Age"], bins, labels=group_names)

age_df = hero_new.groupby(["User Name","Age Group"])
                                                              
age2_df = pd.DataFrame(age_df.size())

age3_df = pd.DataFrame(age2_df.groupby(["Age Group"]).count())
age3_df.columns= ["Total Count"]
age3_df["Percentage of Players"] = 100 * age3_df["Total Count"] / age3_df["Total Count"].sum()
age3_df["Percentage of Players"] = age3_df["Percentage of Players"].map('{:.2f}%'.format)

age3_df

,Total Count,Percentage of Players
Age Group,,
Under 18,108,18.75%
18-25,339,58.85%
26-34,86,14.93%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [27]:
hero_new["Age Group"] = pd.cut(hero_new["Age"], bins, labels=group_names)

age_sales = hero_new.groupby(["Age Group"]).sum()["Price"].rename("Total Purchase Value")
avg_age = hero_new.groupby(["Age Group"]).mean()["Price"].rename("Average Purchase Price")
age_ct = hero_new.groupby(["Age Group"]).count()["Price"].rename("Purchase Count")

ct_total = age_sales / age3_df["Total Count"]

age_df4 = pd.DataFrame({"Purchase Count": age_ct, "Average Purchase Price": avg_age, "Total Purchase Value": age_sales, "Normalized Totals": ct_total})

age_df4["Average Purchase Price"] = age_df4["Average Purchase Price"].map("${:,.2f}".format)
age_df4["Total Purchase Value"] = age_df4["Total Purchase Value"].map("${:,.2f}".format)
age_df4 ["Purchase Count"] = age_df4["Purchase Count"].map("{:,}".format)
age_df4["Normalized Totals"] = age_df4["Normalized Totals"].map("${:,.2f}".format)
age_df4 = age_df4.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

age_df4

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Group,,,,
Under 18,138,$3.05,$420.60,$3.89
18-25,473,$3.06,"$1,448.16",$4.27
26-34,115,$2.83,$325.10,$3.78
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [28]:
sale_sum = hero_new.groupby(["User Name"]).sum()["Price"].rename("Total Purchase Value")
sale_avg = hero_new.groupby(["User Name"]).mean()["Price"].rename("Average Purchase Price")
sale_count = hero_new.groupby(["User Name"]).count()["Price"].rename("Purchase Count")

sale_df = pd.DataFrame({"Total Purchase Value": sale_sum, "Average Purchase Price": sale_avg, "Purchase Count": sale_count})

sale_df["Average Purchase Price"] = sale_df["Average Purchase Price"].map("${:,.2f}".format)
sale_df["Total Purchase Value"] = sale_df["Total Purchase Value"].map("${:,.2f}".format)
sale_df = sale_df.loc[:,["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

sale_df.sort_values("Total Purchase Value", ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
User Name,,,
Haillyrgue51,3,$3.17,$9.50
Phistym51,2,$4.75,$9.50
Lamil79,2,$4.64,$9.29
Aina42,3,$3.07,$9.22
Saesrideu94,2,$4.59,$9.18


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [29]:
sellers = hero_new.loc[:,["Item ID", "Item Name", "Price"]]

total_sellers = sellers.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
avg_buy = sellers.groupby(["Item ID", "Item Name"]).mean()["Price"]
sale_ct = sellers.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")

sale_df = pd.DataFrame({"Total Purchase Value": total_sellers, "Item Price": avg_buy, "Purchase Count": sale_ct})
sale_df["Item Price"] = sale_df["Item Price"].map("${:,.2f}".format)
sale_df["Total Purchase Value"] = sale_df["Total Purchase Value"].map("${:,.2f}".format)
sale_df = sale_df.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

sale_df.sort_values("Purchase Count", ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [30]:
sale_df.sort_values("Total Purchase Value", ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
1,Crucifer,3,$3.26,$9.78
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
